In [3]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from fr_performance import get_fr_detection_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration
from constants import CATEGORICAL_PALETTE


In [4]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
# for id in overview.keys():
#     del overview[id]["SNBm"]
#     del overview[id]["UALm"]
#     del overview[id]["UARm"]


### Activity-specific only FR analysis

In [5]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MCH', 'SQT', 'MIXB', 'STNB', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [6]:
performance_all_participants = get_fr_detection_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAG,1.02,0.97,0.99,-0.08 $\pm$ 0.43,-1 %
1,SNB,PZT,0.98,0.97,0.94,0.56 $\pm$ 0.64,12 %
2,SGB,MAG,1.01,0.93,0.94,-0.24 $\pm$ 0.97,-2 %
3,SGB,PZT,1.38,0.70,0.97,0.53 $\pm$ 1.59,5 %
4,MCH,MAG,1.16,0.85,0.98,-0.12 $\pm$ 0.4,-3 %
5,MCH,PZT,1.03,0.86,0.89,0.19 $\pm$ 0.96,4 %
6,SQT,MAG,1.11,0.89,0.99,-0.0 $\pm$ 0.39,0 %
7,SQT,PZT,1.11,0.82,0.92,0.22 $\pm$ 0.73,6 %
8,MIXB,MAG,0.98,0.99,0.97,-0.1 $\pm$ 0.4,-2 %
9,MIXB,PZT,0.90,0.96,0.86,0.44 $\pm$ 0.91,6 %


In [7]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,Ratio,Precision,Recall,Activity
0,1.040816,1.000000,1.053191,SNB
1,0.731884,1.328571,0.969072,SGB
2,1.126214,0.988372,1.101124,MCH
3,1.000000,1.085366,1.076087,SQT
4,1.088889,1.031250,1.127907,MIXB
5,1.040000,1.000000,1.020619,STNB
6,1.028846,1.011111,1.031915,AAL
7,0.971154,1.113636,1.076087,AAR
8,1.082474,1.068182,1.152941,ALL
9,1.202381,1.104651,1.315068,ALR


In [8]:
mag["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
mag

,Ratio,Precision,Recall,Activity
0,1.02,0.97,0.99,SNB
1,1.01,0.93,0.94,SGB
2,1.16,0.85,0.98,MCH
3,1.11,0.89,0.99,SQT
4,0.98,0.99,0.97,MIXB
5,1.04,0.96,0.99,STNB
6,1.07,0.91,0.97,AAL
7,1.01,0.98,0.99,AAR
8,1.05,0.94,0.98,ALL
9,1.01,0.95,0.96,ALR


### Participant-specific only FR analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [10]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAG,1.09,0.91,1.00,-0.08 $\pm$ 0.38,-1 %
1,7OYX,PZT,1.18,0.78,0.93,-0.13 $\pm$ 1.06,-2 %
2,NO15,MAG,1.13,0.88,0.99,-0.13 $\pm$ 0.4,-2 %
3,NO15,PZT,1.20,0.81,0.98,0.21 $\pm$ 0.57,5 %
4,G8B7,MAG,1.13,0.83,0.93,-0.22 $\pm$ 0.56,-6 %
5,G8B7,PZT,0.95,0.99,0.94,0.37 $\pm$ 0.32,12 %
6,EPE2,MAG,0.98,0.97,0.95,0.16 $\pm$ 0.48,3 %
7,EPE2,PZT,1.04,0.89,0.92,0.14 $\pm$ 0.85,3 %
8,HAK8,MAG,1.11,0.89,0.98,-0.1 $\pm$ 0.37,-2 %
9,HAK8,PZT,1.02,0.77,0.79,0.06 $\pm$ 0.89,2 %


### Overall FR Analysis

In [11]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [12]:
get_fr_detection_performance(overview_all, target=None)

,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,MAG,1.10,0.89,0.98,-0.08 $\pm$ 0.45,-2 %
1,PZT,1.02,0.86,0.88,0.36 $\pm$ 0.87,9 %


### Overall delay analysis

In [13]:
delays_i_all_mag, delays_e_all_mag = [], []
delays_i_all_pzt, delays_e_all_pzt = [], []

for activity in overview_all_participants.keys():

    delays_i_all_mag += overview_all_participants[activity]["MAG"]["delay_i"] 
    delays_e_all_mag += overview_all_participants[activity]["MAG"]["delay_e"]

    delays_i_all_pzt += overview_all_participants[activity]["PZT"]["delay_i"]
    delays_e_all_pzt += overview_all_participants[activity]["PZT"]["delay_e"]

In [14]:
print(f"min delay mag {min(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_i_all_mag + delays_e_all_mag), 2)} +/- {np.round(np.std(delays_i_all_mag + delays_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_i_all_pzt + delays_e_all_pzt), 2)} +/- {np.round(np.std(delays_i_all_pzt + delays_e_all_pzt), 2)}")

min delay mag -0.0
min delay pzt -0.0
max delay mag -2.93
max delay pzt 4.6000000000000005
mean delay mag -0.08 +/- 0.45
mean delay pzt 0.36 +/- 0.87


In [15]:
normality_test(delays_i_all_mag + delays_e_all_mag, sensor="MAG", type="inspiration+expiration")
normality_test(delays_i_all_pzt + delays_e_all_pzt, sensor="PZT", type="inspiration+expiration")

### Event-specific delay analysis

In [16]:
normality_test(delays_i_all_mag, sensor="MAG", type="inspiration")
normality_test(delays_e_all_mag, sensor="MAG", type="expiration")
normality_test(delays_i_all_pzt, sensor="PZT", type="inspiration", categorical_palette=CATEGORICAL_PALETTE[2:])
normality_test(delays_e_all_pzt, sensor="PZT", type="expiration", categorical_palette=CATEGORICAL_PALETTE[2:])

In [17]:
delays_df = pd.DataFrame(columns=["Sensor", "Type", "Delay"])
delays_df["Delay"] = delays_i_all_mag + delays_e_all_mag + delays_i_all_pzt + delays_e_all_pzt
delays_df["Sensor"] = ["MAG"] * len(delays_i_all_mag + delays_e_all_mag) + ["PZT"] * len(delays_i_all_pzt + delays_e_all_pzt)
delays_df["Type"] = ["Inhalation"] * len(delays_i_all_mag) + ["Exhalation"] * len(delays_e_all_mag) + ["Inhalation"] * len(delays_i_all_pzt) + ["Exhalation"] * len(delays_e_all_pzt)
delays_df


,Sensor,Type,Delay
0,MAG,Inhalation,-0.22
1,MAG,Inhalation,-0.05
2,MAG,Inhalation,-0.11
3,MAG,Inhalation,0.01
4,MAG,Inhalation,0.12
...,...,...,...
13441,PZT,Exhalation,0.94
13442,PZT,Exhalation,-0.31
13443,PZT,Exhalation,-0.03
13444,PZT,Exhalation,0.37


In [18]:
plot_inspiration_vs_expiration(delays_df)

#### Statistical testing: comparing inhalation vs expiration

In [19]:
print(scipy.stats.mannwhitneyu(delays_i_all_mag, delays_e_all_mag))
print(scipy.stats.mannwhitneyu(delays_i_all_pzt, delays_e_all_pzt))

MannwhitneyuResult(statistic=6044299.0, pvalue=0.014866451571431051)
MannwhitneyuResult(statistic=5001118.5, pvalue=0.3022400499560812)


In [20]:
print(len(delays_i_all_mag + delays_e_all_mag))
print(len(delays_i_all_pzt + delays_e_all_pzt))

7073
6373


### Correlation between strap-band perimeter and FR performance

In [21]:
strap_thorax_ratio_dict = {
    '7OYX': 0.94,
    'NO15': 0.99,
    'G8B7': 0.96,
    'EPE2': 0.93,
    'HAK8': 0.94,
    '1BST': 0.98,
    '83J1': 0.97,
    'QMQ7': 0.96,
    '9TUL': 0.9,
    'FTD7': 0.9,
    'Y6O3': 0.92,
    '2QWT': 0.85,
    'F9AF': 0.87,
    'P4W9': 0.96,
    'W8Z9': 0.94,
    'D4GQ': 0.92
}

In [22]:
strap_thorax_performance_df = get_fr_detection_performance(overview, target="both")
strap_thorax_performance_df = strap_thorax_performance_df[strap_thorax_performance_df["Sensor"] == "MAG"]
strap_thorax_performance_df['strap-to-thorax ratio'] = strap_thorax_performance_df["ID"].apply(lambda id: strap_thorax_ratio_dict[id])
strap_thorax_performance_df


,ID,Activity,Sensor,Ratio,Precision,Recall,strap-to-thorax ratio
0,7OYX,SNB,MAG,1.00,1.00,1.00,0.94
2,7OYX,SGB,MAG,1.00,1.00,1.00,0.94
4,7OYX,MCH,MAG,0.95,1.00,0.95,0.94
6,7OYX,SQT,MAG,1.04,0.96,1.00,0.94
8,7OYX,MIXB,MAG,1.00,1.00,1.00,0.94
...,...,...,...,...,...,...,...
470,D4GQ,UAL,MAG,2.55,0.39,1.00,0.92
472,D4GQ,UAR,MAG,1.05,0.91,0.95,0.92
474,D4GQ,SE,MAG,1.33,0.75,1.00,0.92
476,D4GQ,SS,MAG,3.00,0.33,1.00,0.92


In [23]:
scipy.stats.spearmanr(strap_thorax_performance_df["Ratio"].values, strap_thorax_performance_df["strap-to-thorax ratio"].values)

SignificanceResult(statistic=0.08976044508908099, pvalue=0.16571563754275126)

### Analysis on alternative positioning

In [24]:
overview_middle_all_participants = transform_overview_on_target(overview_middle, target="Activity")
performance_middle_all_participants = get_fr_detection_performance(overview_middle_all_participants, target="Activity")
performance_middle_all_participants = performance_middle_all_participants[performance_middle_all_participants["Sensor"]=="MAG"]
performance_middle_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay


In [25]:
performance_middle_vs_left = pd.DataFrame(columns=performance_middle_all_participants.columns)

for activity in performance_middle_all_participants["Activity"].unique():
    new_entry = performance_all_participants[(performance_all_participants["Activity"]==activity[:-1]) & (performance_all_participants["Sensor"]=="MAG")]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

    new_entry = performance_middle_all_participants[performance_middle_all_participants["Activity"]==activity]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

performance_middle_vs_left.drop(columns=["Sensor", "Mean absolute delay $\pm$ SD", "Adjusted delay"], inplace=True)
performance_middle_vs_left

,Activity,Ratio,Precision,Recall


In [26]:
delays_middle_i_all_mag, delays_middle_e_all_mag = [], []
delays_middle_i_all_pzt, delays_middle_e_all_pzt = [], []

for activity in overview_middle_all_participants.keys():

    delays_middle_i_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_i"] 
    delays_middle_e_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_e"]

    delays_middle_i_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_i"]
    delays_middle_e_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_e"]

print(f"min delay mag {min(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)} +/- {np.round(np.std(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)} +/- {np.round(np.std(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)}")

ValueError: min() arg is an empty sequence